In [23]:
import pandas as pd
import os
import datetime
import preprocessing_module as ppm
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import pickle as pkl

In [24]:
katse_nr = 3
order = 2
dx = 5

path = '/Users/svennomm/kohalikTree/Data/AIRSCS/wave/data_v2/'
winx=256

In [25]:
path = path + "katse_0" + str(katse_nr) + "/"
initial_data_file_1 = path + 'sarspec_hgh_order_' + str(order) + '_winx_' + str(winx) + '_co_clean.csv'
initial_data_file_2 = path + 'sarspec_hgh_order_' + str(order) + '_winx_' + str(winx) + '_cro_clean.csv'
target_data_file = path + 'wavespec_hgh_order_' + str(order) + '_winx_' + str(winx) + '_clean.csv'

In [26]:
initial_data_1 = pd.read_csv(initial_data_file_1, sep=',')
initial_data_2 = pd.read_csv(initial_data_file_2, sep=',')
target_data = pd.read_csv(target_data_file, sep=',')

In [ ]:
initial_data_1

In [ ]:
initial_data_2

In [ ]:
target_data

In [30]:
input_data_1_train, input_data_2_train, target_data_train, input_data_1_test, input_data_2_test, target_data_test,\
        test_data_indexes = ppm.time_based_splitter(initial_data_1, initial_data_2, target_data)

In [ ]:
input_data_1_train

In [ ]:
input_data_2_train

In [ ]:
target_data_train

In [ ]:
input_data_1_test

In [ ]:
input_data_1_train, input_data_2_train, target_data_train = ppm.initial_formatting_old_data(input_data_1_train, input_data_2_train, target_data_train)
input_data_1_test, input_data_2_test, target_data_test = ppm.initial_formatting_old_data(input_data_1_test,input_data_2_test, target_data_test)

In [ ]:
input_data_1_train

In [37]:
device = torch.device('mps')
input_tensor_1_train = torch.tensor(input_data_1_train.values).float()
input_tensor_2_train = torch.tensor(input_data_2_train.values).float()
target_tensor_train = torch.tensor(target_data_train.values).float()

input_tensor_1_test = torch.tensor(input_data_1_test.values).float()
input_tensor_2_test = torch.tensor(input_data_2_test.values).float()
target_tensor_test = torch.tensor(target_data_test.values).float()

In [38]:
print(torch.Tensor.size(input_tensor_1_train),torch.Tensor.size(input_tensor_2_train), torch.Tensor.size(target_tensor_train))
print(torch.Tensor.size(input_tensor_1_test),torch.Tensor.size(input_tensor_2_test), torch.Tensor.size(target_tensor_test))

torch.Size([3868, 69]) torch.Size([3868, 69]) torch.Size([3868, 69])
torch.Size([74, 69]) torch.Size([74, 69]) torch.Size([74, 69])


In [39]:
input_tensor_train_log_div_norm = ppm.combine_polarisations_log_div_norm(input_tensor_1_train, input_tensor_2_train)
input_tensor_train_div_norm = ppm.combine_polarisations_div_norm(input_tensor_1_train, input_tensor_2_train)
input_tensor_train_div = ppm.combine_polarisations_div(input_tensor_1_train, input_tensor_2_train)

input_tensor_test_log_div_norm = ppm.combine_polarisations_log_div_norm(input_tensor_1_test, input_tensor_2_test)
input_tensor_test_div_norm = ppm.combine_polarisations_div_norm(input_tensor_1_test, input_tensor_2_test)
input_tensor_test_div = ppm.combine_polarisations_div(input_tensor_1_test, input_tensor_2_test)

In [40]:
print(torch.Tensor.size(input_tensor_train_div),torch.Tensor.size(input_tensor_train_div_norm), torch.Tensor.size(input_tensor_train_log_div_norm))
print(torch.Tensor.size(input_tensor_test_div),torch.Tensor.size(input_tensor_test_div_norm), torch.Tensor.size(input_tensor_test_log_div_norm))

torch.Size([3868, 69]) torch.Size([3868, 69]) torch.Size([3868, 69])
torch.Size([74, 69]) torch.Size([74, 69]) torch.Size([74, 69])


In [41]:
input_data_train_log_div_norm = input_tensor_train_log_div_norm.detach().cpu().numpy()
input_data_train_div_norm = input_tensor_train_div_norm.detach().cpu().numpy()
input_data_train_div = input_tensor_train_div.detach().cpu().numpy()

target_data_train = target_tensor_train.detach().cpu().numpy()
input_data_1_train = input_tensor_1_train.detach().cpu().numpy()
input_data_2_train = input_tensor_2_train.detach().cpu().numpy()

input_data_test_log_div_norm = input_tensor_test_log_div_norm.detach().cpu().numpy()
input_data_test_div_norm = input_tensor_test_div_norm.detach().cpu().numpy()
input_data_test_div = input_tensor_test_div.detach().cpu().numpy()
target_data_test = target_tensor_test.detach().cpu().numpy()
input_data_1_test = input_tensor_1_test.detach().cpu().numpy()
input_data_2_test = input_tensor_2_test.detach().cpu().numpy()

In [42]:
print(torch.Tensor.size(input_tensor_train_div),torch.Tensor.size(input_tensor_train_div_norm), torch.Tensor.size(input_tensor_train_log_div_norm))
print(torch.Tensor.size(input_tensor_test_div),torch.Tensor.size(input_tensor_test_div_norm), torch.Tensor.size(input_tensor_test_log_div_norm))

torch.Size([3868, 69]) torch.Size([3868, 69]) torch.Size([3868, 69])
torch.Size([74, 69]) torch.Size([74, 69]) torch.Size([74, 69])


In [43]:
input_data_train_log_div_norm_trunc = ppm.cut_tail(input_data_train_log_div_norm, 60)
input_data_train_div_norm_trunc = ppm.cut_tail(input_data_train_div_norm, 60)
input_data_train_div_trunc = ppm.cut_tail(input_data_train_div, 60)
target_data_train_trunc = ppm.cut_tail(target_data_train, 60)

input_data_test_log_div_norm_trunc = ppm.cut_tail(input_data_test_log_div_norm, 60)
input_data_test_div_norm_trunc = ppm.cut_tail(input_data_test_div_norm, 60)
input_data_test_div_trunc = ppm.cut_tail(input_data_test_div, 60)
target_data_test_trunc = ppm.cut_tail(target_data_test, 60)

input_data_1_train_trunc = ppm.cut_tail(input_data_1_train, 60)
input_data_2_train_trunc = ppm.cut_tail(input_data_2_train, 60)
input_data_1_test_trunc = ppm.cut_tail(input_data_1_test, 60)
input_data_2_test_trunc = ppm.cut_tail(input_data_2_test, 60)

In [44]:
print(type(input_data_train_div), input_data_train_div.shape)
print(type(input_data_train_div_norm), input_data_train_div_norm.shape)
print(type(input_data_train_log_div_norm), input_data_train_log_div_norm.shape)
print(type(target_data_train), target_data_train.shape)

print(type(input_data_test_div), input_data_test_div.shape)
print(type(input_data_test_div_norm), input_data_test_div_norm.shape)
print(type(input_data_test_log_div_norm), input_data_test_log_div_norm.shape)
print(type(target_data_test), target_data_test.shape)

<class 'numpy.ndarray'> (3868, 69)
<class 'numpy.ndarray'> (3868, 69)
<class 'numpy.ndarray'> (3868, 69)
<class 'numpy.ndarray'> (3868, 69)
<class 'numpy.ndarray'> (74, 69)
<class 'numpy.ndarray'> (74, 69)
<class 'numpy.ndarray'> (74, 69)
<class 'numpy.ndarray'> (74, 69)


In [45]:
#generate file name
path = '/Users/svennomm/kohalikTree/Data/AIRSCS/wave/data_v2/'
path_1 = path + 'alternative_processing/' + "katse_0" + str(katse_nr) + '/'
fname_train = path_1 + "katse_0" + str(katse_nr) + '_winx_' + str(winx) + '_ldn' + '.pkl'

data = [input_data_train_log_div_norm,  input_data_test_log_div_norm, target_data_train, target_data_test, test_data_indexes]
with open(fname_train, 'wb') as f:
    pkl.dump(data, f)
    f.close()


fname_train = path_1 + "katse_0" + str(katse_nr) + '_winx_' + str(winx) + '_dn' + '.pkl'

data = [input_data_train_div_norm, input_data_test_div_norm, target_data_train, target_data_test, test_data_indexes]
with open(fname_train, 'wb') as f:
    pkl.dump(data, f)
    f.close()


fname_train = path_1 + "katse_0" + str(katse_nr) + '_winx_' + str(winx) + '_ldnt' + '.pkl'

data = [input_data_train_log_div_norm_trunc, input_data_test_log_div_norm_trunc, target_data_train_trunc, target_data_test_trunc, test_data_indexes]
with open(fname_train, 'wb') as f:
    pkl.dump(data, f)
    f.close()

fname_train = path_1 + "katse_0" + str(katse_nr) + '_winx_' + str(winx) + '_dnt' + '.pkl'


data = [input_data_train_div_norm_trunc, input_data_test_div_norm_trunc, target_data_train_trunc, target_data_test_trunc, test_data_indexes]
with open(fname_train, 'wb') as f:
    pkl.dump(data, f)
    f.close()


fname_train = path_1 + "katse_0" + str(katse_nr) + '_winx_' + str(winx) + '_d' + '.pkl'


data = [input_data_train_div, input_data_test_div, target_data_train, target_data_test, test_data_indexes]
with open(fname_train, 'wb') as f:
    pkl.dump(data, f)
    f.close()
    

fname_train = path_1 + "katse_0" + str(katse_nr) + '_winx_' + str(winx) + '_dt' + '.pkl'


data = [input_data_train_div_trunc, input_data_test_div_trunc, target_data_train_trunc, target_data_test_trunc, test_data_indexes]
with open(fname_train, 'wb') as f:
    pkl.dump(data, f)
    f.close()

fname_train = path_1 + "katse_0" + str(katse_nr) + '_winx_' + str(winx) + '_o' + '.pkl'


data = [input_data_1_train, input_data_1_test, input_data_2_train, input_data_2_test, target_data_train, target_data_test, test_data_indexes]
with open(fname_train, 'wb') as f:
    pkl.dump(data, f)
    f.close()

fname_train = path_1 + "katse_0" + str(katse_nr) + '_winx_' + str(winx) + '_ot' + '.pkl'


data = [input_data_1_train_trunc, input_data_1_test_trunc, input_data_2_train_trunc, input_data_2_test_trunc, target_data_train_trunc, target_data_test_trunc, test_data_indexes]
with open(fname_train, 'wb') as f:
    pkl.dump(data, f)
    f.close()